In [42]:
ORIGINAL_DATA_FILE ='[new] yancheng_train_20171226_2.csv'
AVERGEVALUE_COLUMNS = ['price_level', 'rated_passenger']
ONEHOT_SPECILA_COLUMNS = ['gearbox_type']

DROPPED_COLUMNS = ['price']
MAP_COLUMNS = ['if_charging']
SUBONE_COLUMNS = ['if_luxurious_id', 'if_MPV_id']
SPLIT_COLUMNS = ['sale_date']
OBJECT_COLUMNS = ['sale_quantity']

# predict to fill the blank data
PREDICT_COLUMNS = ['level_id', 'engine_torque']

ONEHOT_COLUMNS=['class_id', 'brand_id', 'type_id', 'department_id', 'gearbox_type',
                'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id']
NORMALIZE_COLUMNS = ['compartment','TR','displacement','price_level','power','cylinder_number',
                  'car_length','car_height','car_width','total_quality','equipment_quality',
                  'rated_passenger','wheelbase','front_track','rear_track'] + PREDICT_COLUMNS

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

In [58]:
df = pd.read_csv('../'+ORIGINAL_DATA_FILE)
df.head()
# for column in df.columns:
#     if column not in ONEHOT_COLUMNS+NORMALIZE_COLUMNS+SPLIT_COLUMNS+MAP_COLUMNS+SUBONE_COLUMNS+OBJECT_COLUMNS+DROPPED_COLUMNS:
#         print(column)

c:\ricy\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,...,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201601,175962,8,750,3,2,1,4,5,MT,...,147.5,4473,1706,1469,1735,1275,5,2603,1460,1500
1,201604,175962,32,750,3,2,1,4,5,MT,...,147.5,4473,1706,1469,1735,1275,5,2603,1460,1500
2,201603,961962,5,750,3,2,1,4,5,MT,...,147.5,4487,1706,1470,1740,1260,5,2603,1460,1500
3,201612,527765,195,236,2,2,5,1,1,AT,...,-,3675,1630,1518,1360,1050,4,2360,1405,1400
4,201704,527765,18,236,2,2,5,1,1,AT,...,-,3675,1630,1518,1360,1058,4,2360,1405,1400


In [59]:

# handle fuelTypeId by hand
# def handleFuelTypeId(df):
#     classDicBasic = {}
#     for index, row in df.iterrows():
#         if row['class_id'] not in classDicBasic:
#             jump = False
#             for subIndex, value in row.items():
#                 if value == '-' and subIndex != 'price':
#                     jump = True
#             if not jump:
#                 classDicBasic[row['class_id']] = row
#     # fill in the value, not include price and 527765, 178529
#     classIds = classDicBasic.keys()
#     for index, row in df.iterrows():
#         classId = row['class_id']
#         if row['fuel_type_id'] == '-':
#             if classId in classIds:
#                 item = classDicBasic[classId]
#                 row['fuel_type_id'] = item['fuel_type_id']
#                 df.set_value(index, 'fuel_type_id', item['fuel_type_id'])
#     return df

def mapPriceLevel(x):
    arr = x.split('-')
    if len(arr) < 2:
        return float(arr[0].split('W')[0])/2
    else:
        subArr = arr[1].split('W')
        return (float(arr[0]) + float(subArr[0]))/2
df['price_level'] = df['price_level'].apply(mapPriceLevel)

def mapRatedPassenger(x):
    monthMap = {'Jul':6,'May':5,'Aug':8}
    if '-' in x:
        arr = x.split('-')
        if arr[1] in monthMap.keys():
            arr[1] = monthMap.get(arr[1])
#             print(arr[1], 'in monthMap')
        return (float(arr[0]) + float(arr[1]))/2
    else:
        return float(x)
df['rated_passenger'] = df['rated_passenger'].apply(mapRatedPassenger)
    
def mapTR(x):
    if type(x)==str and ';' in x:
        nums = x.split(';')
        trNum = (float(nums[0]) + float(nums[1]))/2
        return trNum
    else:
        return float(x)
df['TR'] = df['TR'].apply(mapTR)

mapSuboneColumn = lambda x:x-1
df[SUBONE_COLUMNS] = df[SUBONE_COLUMNS].apply(mapSuboneColumn)

def mapCharging(x):
    if 'L' in x:
        return 1
    else:
        return 0
df['if_charging'] = df['if_charging'].apply(mapCharging)

def handleSaleDate(df):
    # split date to month and year
    def operateDateToYear(x):
        return x[:4]
    def operateDateToMonth(x):
        return x[4:]
    df['sale_date'] = df['sale_date'].astype('str')
    df['year'] = df['sale_date'].apply(operateDateToYear)
    df['month'] = df['sale_date'].apply(operateDateToMonth)
    df = df.drop(['sale_date'], axis=1)
    return df
df = handleSaleDate(df)

def mapPower(x):
    if isinstance(x, str):
        if '/' in x:
            arr = x.split('/')
            num = (float(arr[0])+float(arr[1]))/2
            return num
        elif x != '-':
            return float(x)
    return x
df['power'] = df['power'].apply(mapPower)

df = pd.get_dummies(df, columns=['year', 'month', 'class_id', 'brand_id', 'type_id', 'department_id', 'gearbox_type',
                             'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id'])

def handleGeabox(df):
    geaboxDf = pd.read_csv('../gearbox_type_one_hot.csv')
    df = df.drop(['gearbox_type_AMT', 'gearbox_type_AT', 'gearbox_type_AT;DCT', 'gearbox_type_CVT', 'gearbox_type_DCT',
                  'gearbox_type_MT', 'gearbox_type_MT;AT', 'price'], axis = 1)
    df = pd.concat([df, geaboxDf], axis=1)
    return df 
df = handleGeabox(df)


In [60]:
# handle 155/140 by hand first
def handlePredictColumns(df):
    # used below columns to predict
    levelIdInput = ['gearbox_type', 'driven_type','fuel_type','newenergy_type','emission_standards']
    engineTorqueInput = ['if_charging', 'car_length','displacement','wheelbase','rear_track','if_luxurious_id',
                    'front_track','car_width','total_quality','price_level','equipment_quality','power']
    
    levelIdInputColumnList = []
    engineTorqueInputColumnList = []
    for column in df.columns.values:
        for levelInput in levelIdInput:
            if levelInput in column:
                levelIdInputColumnList.append(column)
        for engineInput in engineTorqueInput:
            if engineInput in column:
                engineTorqueInputColumnList.append(column)
                
    # produce the train input and output           
    level_id_exception_condition = df['level_id'] == '-'
    engine_torque_exception_condition = df['engine_torque'] == '-'
    trainDf = df.drop(df[level_id_exception_condition | engine_torque_exception_condition].index)
    levelIdPredictDf = df[level_id_exception_condition]
    engineTorquePredictDf = df[engine_torque_exception_condition]
    
    levelIdInputData = trainDf.loc[:, levelIdInputColumnList]
    levelIdOutputData = trainDf.loc[:, 'level_id']
    engineTorqueInputData = trainDf.loc[:, engineTorqueInputColumnList]
    engineTorqueOutputData = trainDf.loc[:, 'engine_torque']
    
    # train input and output
    levelIdTrainInput = levelIdInputData.values
    levelIdTrainOutput = levelIdOutputData.values
    engineTorqueInput = engineTorqueInputData.values
    engineTorqueOutput = engineTorqueOutputData.values

    levelIdModel = linear_model.LinearRegression() # model for levelId 
    engineTorqueModel = linear_model.LinearRegression() # model for engineTorque
    
    # train model
    levelIdModel.fit(levelIdTrainInput, levelIdTrainOutput)
    engineTorqueModel.fit(engineTorqueInput, engineTorqueOutput)
    
    # produce predict input and output
    levelIdpredictIndex = levelIdPredictDf.index
    levelIdpredictInput = levelIdPredictDf.loc[:, levelIdInputColumnList]
    engineTorquePredictIndex = engineTorquePredictDf.index
    engineTorquePredictInput = engineTorquePredictDf.loc[:, engineTorqueInputColumnList]
    
    levelIdPredictResult = levelIdModel.predict(levelIdpredictInput.values)
    engineTorquePredictResult = engineTorqueModel.predict(engineTorquePredictInput.values)

    levelIdIndexCount = 0
    engineIndexCount = 0 
    for levelIndex in levelIdpredictIndex:
        df.loc[levelIndex, 'level_id'] = round(levelIdPredictResult[levelIdIndexCount])
        levelIdIndexCount = levelIdIndexCount + 1

    for engineIndex in engineTorquePredictIndex:
        df.loc[engineIndex, 'engine_torque'] = engineTorquePredictResult[engineIndexCount]
        engineIndexCount = engineIndexCount + 1
    
    return df
df = handlePredictColumns(df)

In [62]:
df.to_csv('../trainDataAfterClean.csv', encoding='utf-8', index=False)